In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tensorflow import keras
from tensorflow.keras import layers
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataisbeautiful/r_dataisbeautiful_posts.csv


Using TensorFlow backend.


In [2]:
data = pd.read_csv("/kaggle/input/dataisbeautiful/r_dataisbeautiful_posts.csv")
print(data.shape)
data.head()

(183891, 12)


,id,title,score,author,author_flair_text,removed_by,total_awards_received,awarders,created_utc,full_link,num_comments,over_18
0,hbyv2e,[OC] How Bar Chart Race Animation Can be Creat...,1,Tom10AE,NaN,NaN,0.0,[],1592565317,https://www.reddit.com/r/dataisbeautiful/comme...,0,False
1,hbynoy,[UK] Labour and Conservative vote share by age...,1,Ge0rgeBr0ughton,NaN,NaN,0.0,[],1592564316,https://www.reddit.com/r/dataisbeautiful/comme...,1,False
2,hbym2y,[NOC] Number of wallets returned across the wo...,1,i_am_shattered,NaN,NaN,0.0,[],1592564089,https://www.reddit.com/r/dataisbeautiful/comme...,1,False
3,hbyirw,Largest Banana Producing Countries (1961-2020)...,1,Bilal078692,NaN,moderator,0.0,[],1592563628,https://www.reddit.com/r/dataisbeautiful/comme...,0,False
4,hby1fy,[OC] Popular Combinations for the StamiStudios...,1,shahinrostami,Viz Researcher,NaN,0.0,[],1592561119,https://www.reddit.com/r/dataisbeautiful/comme...,2,False


In [3]:
fig = px.bar(x=data.over_18.value_counts().index,
             y=data.over_18.value_counts().values,
            color=data.over_18.value_counts().values,
            )
fig.update_layout(title_text="Distrubition of Text")
fig.show()

In [4]:
data = data[["title","score","num_comments","over_18"]]
data.head()

,title,score,num_comments,over_18
0,[OC] How Bar Chart Race Animation Can be Creat...,1,0,False
1,[UK] Labour and Conservative vote share by age...,1,1,False
2,[NOC] Number of wallets returned across the wo...,1,1,False
3,Largest Banana Producing Countries (1961-2020)...,1,0,False
4,[OC] Popular Combinations for the StamiStudios...,1,2,False


In [5]:
data = data.dropna()

In [6]:
from bs4 import BeautifulSoup
data['text_clean'] = data['title'].apply(lambda x: " ".join(x.lower() for x in x.split()))

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text


print("Data cleaning in progress...")
data["text_clean"] = data["text_clean"].apply(denoise_text)

Data cleaning in progress...


In [7]:
stopwords_set= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","im","youre","I'd","I","Shes","she's","She's","but","But","its","shes","hes","Hes"])

x = stopwords.words("english")
stopwords_nltk_en = set(x)
stoplist_combined = set.union(stopwords_set, stopwords_nltk_en)

In [8]:
import string
def clean_text(text):
    text = text.lower()
    text = re.sub("\[.*?\]", "", text)
    text = re.sub("https?://\S|www\.\S+", "", text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def text_preprocessing(text):
    tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    remove_stopwords = [w for w in tokenized_text if w not in stoplist_combined]
    combined_text = ' '.join(remove_stopwords)
    return combined_text


data["text_clean"] = data["text_clean"].apply(lambda x:text_preprocessing(x))

In [9]:
data["text_clean"] = data["text_clean"].apply(lambda x: word_tokenize(x))

## With Bidirectional LSTM

In [10]:
max_features = 20000 # en iyi 20000 kelime
maxlen = 200  # her bir metinden 200 kelime

inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,757,761
Trainable params: 2,757,761
Non-trainable params: 0
_________________________________________________________________


In [11]:
data["over_18"] = [0 if i==False else 1 for i in data["over_18"]]

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=2000, split=" ")
tokenizer.fit_on_texts(data["text_clean"])

X = tokenizer.texts_to_sequences(data['text_clean'])
X = pad_sequences(X)

y = data["over_18"]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=35)

In [14]:
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

In [25]:
# dengesiz dağılıma sahip sınıflar olduğu için en iyi değerlendirme metriği olarak AUC'ı seçiyorum
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=keras.metrics.AUC())
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
2155/2155 [==============================] - 145s 67ms/step - loss: 0.0300 - auc_1: 0.6644 - val_loss: 0.0263 - val_auc_1: 0.7627
Epoch 2/10
2155/2155 [==============================] - 145s 67ms/step - loss: 0.0226 - auc_1: 0.8461 - val_loss: 0.0267 - val_auc_1: 0.7864
Epoch 3/10
2155/2155 [==============================] - 143s 66ms/step - loss: 0.0195 - auc_1: 0.9181 - val_loss: 0.0307 - val_auc_1: 0.7315
Epoch 4/10
2155/2155 [==============================] - 144s 67ms/step - loss: 0.0168 - auc_1: 0.9379 - val_loss: 0.0346 - val_auc_1: 0.6994
Epoch 5/10
2155/2155 [==============================] - 143s 66ms/step - loss: 0.0147 - auc_1: 0.9492 - val_loss: 0.0343 - val_auc_1: 0.7011
Epoch 6/10
2155/2155 [==============================] - 143s 66ms/step - loss: 0.0131 - auc_1: 0.9591 - val_loss: 0.0383 - val_auc_1: 0.6993
Epoch 7/10
2155/2155 [==============================] - 142s 66ms/step - loss: 0.0116 - auc_1: 0.9605 - val_loss: 0.0394 - val_auc_1: 0.6688
Epoch 8/10
21